In [8]:
import pandas as pd

# 读取上传的 CSV 文件
file_path = '/Users/fenglingyi/Downloads/groupwork_final/property_addresses_to_lat_lng_2_final.csv'
df = pd.read_csv(file_path)

# 计算每份的大小
batch_size = len(df) // 5

# 分割数据为五份
df_parta = df.iloc[:batch_size]
df_partb = df.iloc[batch_size:batch_size*2]
df_partc = df.iloc[batch_size*2:batch_size*3]
df_partd = df.iloc[batch_size*3:batch_size*4]
df_parte = df.iloc[batch_size*4:]

# 保存每个部分为新的 CSV 文件
output_path_parta = '/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parta.csv'
output_path_partb = '/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partb.csv'
output_path_partc = '/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partc.csv'
output_path_partd = '/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partd.csv'
output_path_parte = '/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parte.csv'

# 保存到 CSV 文件中
df_parta.to_csv(output_path_parta, index=False)
df_partb.to_csv(output_path_partb, index=False)
df_partc.to_csv(output_path_partc, index=False)
df_partd.to_csv(output_path_partd, index=False)
df_parte.to_csv(output_path_parte, index=False)

print(f"文件已分割并保存为: {output_path_parta}, {output_path_partb}, {output_path_partc}, {output_path_partd}, 和 {output_path_parte}")


文件已分割并保存为: /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parta.csv, /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partb.csv, /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partc.csv, /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partd.csv, 和 /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parte.csv


<p style="font-size:24px;">A</p>


In [11]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# 读取房屋和车站的 CSV 文件
houses_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parta.csv')  # 替换为你的房屋CSV文件路径
schools_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_file/victoria_schools.csv', encoding='ISO-8859-1')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


# 创建 GeoJSON Feature 的函数
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # 如果没有 'address' 列，请根据你的实际列修改
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Features
schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


# 保存房屋和车站分别为 GeoJSON 文件
houses_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_a.geojson'
schools_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("房屋和车站的GeoJSON文件已生成")


房屋和车站的GeoJSON文件已生成


In [14]:
import geojson
import openrouteservice
import pandas as pd
import time

# 初始化 OpenRouteService API 客户端
api_key = '5b3ce3597851110001cf624880467d82d87348ab87a207c27ad5516c'  # 请替换为你的 OpenRouteService API Key
client = openrouteservice.Client(key=api_key)

# 读取 GeoJSON 文件
with open('/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_a.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

# 提取经纬度
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

# 提取房屋和车站的经纬度
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

# 使用 OpenRouteService API 批量计算距离并按批次处理
def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # 处理每个房屋和车站的距离，找出最短的距离
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # 排除 None 值的距离，找出最小距离
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

# 分批处理
shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

# 保存最短距离结果为 CSV 文件
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '/Users/fenglingyi/Downloads/groupwork_generate/a_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"结果已保存到 {output_path}")


结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/a_closest_school_distance_final_2.csv


<p style="font-size:24px;">B</p>


In [1]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# 读取房屋和车站的 CSV 文件
houses_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partb.csv')  # 替换为你的房屋CSV文件路径
schools_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_file/victoria_schools.csv', encoding='ISO-8859-1')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


# 创建 GeoJSON Feature 的函数
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # 如果没有 'address' 列，请根据你的实际列修改
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Features
schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


# 保存房屋和车站分别为 GeoJSON 文件
houses_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_b.geojson'
schools_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("房屋和车站的GeoJSON文件已生成")


房屋和车站的GeoJSON文件已生成


In [2]:
import geojson
import openrouteservice
import pandas as pd
import time

# 初始化 OpenRouteService API 客户端
api_key = '5b3ce3597851110001cf624880467d82d87348ab87a207c27ad5516c'  # 请替换为你的 OpenRouteService API Key
client = openrouteservice.Client(key=api_key)

# 读取 GeoJSON 文件
with open('/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_b.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

# 提取经纬度
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

# 提取房屋和车站的经纬度
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

# 使用 OpenRouteService API 批量计算距离并按批次处理
def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # 处理每个房屋和车站的距离，找出最短的距离
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # 排除 None 值的距离，找出最小距离
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

# 分批处理
shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

# 保存最短距离结果为 CSV 文件
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '/Users/fenglingyi/Downloads/groupwork_generate/b_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"结果已保存到 {output_path}")

结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/b_closest_school_distance_final_2.csv


<p style="font-size:24px;">C</p>


In [3]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# 读取房屋和车站的 CSV 文件
houses_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partc.csv')  # 替换为你的房屋CSV文件路径
schools_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_file/victoria_schools.csv', encoding='ISO-8859-1')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


# 创建 GeoJSON Feature 的函数
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # 如果没有 'address' 列，请根据你的实际列修改
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Features
schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


# 保存房屋和车站分别为 GeoJSON 文件
houses_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_c.geojson'
schools_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("房屋和车站的GeoJSON文件已生成")

房屋和车站的GeoJSON文件已生成


In [6]:
import geojson
import openrouteservice
import pandas as pd
import time

# 初始化 OpenRouteService API 客户端
api_key = '5b3ce3597851110001cf6248e145f61cd48746779f79c90e911dfd2b'  # 请替换为你的 OpenRouteService API Key
client = openrouteservice.Client(key=api_key)

# 读取 GeoJSON 文件
with open('/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_c.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

# 提取经纬度
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

# 提取房屋和车站的经纬度
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

# 使用 OpenRouteService API 批量计算距离并按批次处理
def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # 处理每个房屋和车站的距离，找出最短的距离
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # 排除 None 值的距离，找出最小距离
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

# 分批处理
shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

# 保存最短距离结果为 CSV 文件
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '/Users/fenglingyi/Downloads/groupwork_generate/c_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"结果已保存到 {output_path}")

Error calculating distance matrix: 500 ({'error': 'There was a problem proxying the request'})
结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/c_closest_school_distance_final_2.csv


<p style="font-size:24px;">D</p>


In [4]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# 读取房屋和车站的 CSV 文件
houses_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partd.csv')  # 替换为你的房屋CSV文件路径
schools_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_file/victoria_schools.csv', encoding='ISO-8859-1')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


# 创建 GeoJSON Feature 的函数
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # 如果没有 'address' 列，请根据你的实际列修改
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Features
schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


# 保存房屋和车站分别为 GeoJSON 文件
houses_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_d.geojson'
schools_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("房屋和车站的GeoJSON文件已生成")

房屋和车站的GeoJSON文件已生成


In [7]:
import geojson
import openrouteservice
import pandas as pd
import time

# 初始化 OpenRouteService API 客户端
api_key = '5b3ce3597851110001cf62487af3a2624ef84344babc38f1f6fde19c'  # 请替换为你的 OpenRouteService API Key
client = openrouteservice.Client(key=api_key)

# 读取 GeoJSON 文件
with open('/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_d.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

# 提取经纬度
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

# 提取房屋和车站的经纬度
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

# 使用 OpenRouteService API 批量计算距离并按批次处理
def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # 处理每个房屋和车站的距离，找出最短的距离
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # 排除 None 值的距离，找出最小距离
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

# 分批处理
shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

# 保存最短距离结果为 CSV 文件
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '/Users/fenglingyi/Downloads/groupwork_generate/d_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"结果已保存到 {output_path}")

结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/d_closest_school_distance_final_2.csv


<p style="font-size:24px;">E</p>


In [5]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# 读取房屋和车站的 CSV 文件
houses_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parte.csv')  # 替换为你的房屋CSV文件路径
schools_df = pd.read_csv('/Users/fenglingyi/Downloads/groupwork_file/victoria_schools.csv', encoding='ISO-8859-1')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


# 创建 GeoJSON Feature 的函数
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # 如果没有 'address' 列，请根据你的实际列修改
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Features
schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


# 保存房屋和车站分别为 GeoJSON 文件
houses_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_e.geojson'
schools_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("房屋和车站的GeoJSON文件已生成")

房屋和车站的GeoJSON文件已生成


In [8]:
import geojson
import openrouteservice
import pandas as pd
import time

# 初始化 OpenRouteService API 客户端
api_key = '5b3ce3597851110001cf62485f72fc1cea314a03ad514d2a07be9533'  # 请替换为你的 OpenRouteService API Key
client = openrouteservice.Client(key=api_key)

# 读取 GeoJSON 文件
with open('/Users/fenglingyi/Downloads/groupwork_generate/property_geojson_e.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('/Users/fenglingyi/Downloads/groupwork_generate/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

# 提取经纬度
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

# 提取房屋和车站的经纬度
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

# 使用 OpenRouteService API 批量计算距离并按批次处理
def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # 处理每个房屋和车站的距离，找出最短的距离
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # 排除 None 值的距离，找出最小距离
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

# 分批处理
shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

# 保存最短距离结果为 CSV 文件
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '/Users/fenglingyi/Downloads/groupwork_generate/e_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"结果已保存到 {output_path}")

结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/e_closest_school_distance_final_2.csv


In [9]:
import pandas as pd

# 定义每个文件的路径
file1 = '/Users/fenglingyi/Downloads/groupwork_generate/a_closest_school_distance_final_2.csv'
file2 = '/Users/fenglingyi/Downloads/groupwork_generate/b_closest_school_distance_final_2.csv'
file3 = '/Users/fenglingyi/Downloads/groupwork_generate/c_closest_school_distance_final_2.csv'
file4 = '/Users/fenglingyi/Downloads/groupwork_generate/d_closest_school_distance_final_2.csv'
file5 = '/Users/fenglingyi/Downloads/groupwork_generate/e_closest_school_distance_final_2.csv'

# 读取每个文件
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)
df5 = pd.read_csv(file5)

# 合并所有数据框
merged_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# 保存合并后的数据框为新的 CSV 文件
output_file = '/Users/fenglingyi/Downloads/groupwork_final/closest_school_distance_final_2.csv'
merged_df.to_csv(output_file, index=False)

print(f"所有文件已合并并保存为: {output_file}")


所有文件已合并并保存为: /Users/fenglingyi/Downloads/groupwork_final/closest_school_distance_final_2.csv
